### Домашнее задание №2 к теме "Spark SQL"

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as f # импортируем функции
import warnings # отключаем предупреждения анаконды
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder.appName('df-project').getOrCreate()

22/07/23 22:48:43 WARN Utils: Your hostname, MacBook-Air-Victor.local resolves to a loopback address: 127.0.0.1; using 192.168.31.211 instead (on interface en0)
22/07/23 22:48:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/23 22:48:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/23 22:48:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/23 22:48:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/07/23 22:48:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/07/23 22:48:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
spark

In [4]:
df = spark.read.\
option('inferSchema', True).\
option('sep', ',').\
option('header', True).\
csv('Downloads/covid-data.csv')

In [5]:
windowSpec = Window()

In [6]:
df.printSchema()

root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- total_cases: double (nullable = true)
 |-- new_cases: double (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: double (nullable = true)
 |-- new_deaths: double (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: double (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: double (nullable = true)
 |-- hosp_patients_per_mill

#### Задание 1  
Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

Проверим какие локации маркируются префиксом __OWID___

In [7]:
df.filter(f.col('iso_code').contains('OWID')).select([
    'iso_code', 
    'location']).distinct().show()

+--------+---------------+
|iso_code|       location|
+--------+---------------+
|OWID_ASI|           Asia|
|OWID_AFR|         Africa|
|OWID_EUR|         Europe|
|OWID_EUN| European Union|
|OWID_INT|  International|
|OWID_KOS|         Kosovo|
|OWID_OCE|        Oceania|
|OWID_NAM|  North America|
|OWID_CYN|Northern Cyprus|
|OWID_SAM|  South America|
|OWID_WRL|          World|
+--------+---------------+



Это континенты, но почету-то в том же списке Косово. Пересохраним для этого задания отфильтрованный датафрейм с нужными столбцами и уже с ним проведем манипуляции.

In [8]:
df_1 = df.filter(
    (f.col('date') <= f.lit('2022-03-31')) & 
    (~f.col('iso_code').contains('OWID')) |
    (f.col('iso_code').contains('KOS'))).select([
    'iso_code', 
    'location', 
    'total_cases', 
    'population'])

In [9]:
df_1.filter(f.col('iso_code').contains('OWID')).select([
    'iso_code', 
    'location']).distinct().show()

+--------+--------+
|iso_code|location|
+--------+--------+
|OWID_KOS|  Kosovo|
+--------+--------+



Косово на месте, континенты удалены

In [10]:
res_1 = df_1.groupBy('iso_code', 'location').agg((f.sum('total_cases')/ f.max('population')).alias('percent')).orderBy(f.col(
    'percent').desc()).limit(15)
res_1 = res_1.withColumn('percent', f.concat(f.round(res_1['percent'], 2), f.lit('%')))

In [11]:
res_1.show()

+--------+-------------+-------+
|iso_code|     location|percent|
+--------+-------------+-------+
|     AND|      Andorra| 23.55%|
|     SMR|   San Marino| 18.83%|
|     MNE|   Montenegro| 17.31%|
|     CZE|      Czechia| 15.78%|
|     QAT|        Qatar| 15.28%|
|     BHR|      Bahrain|  15.2%|
|     LUX|   Luxembourg| 14.86%|
|     USA|United States| 14.13%|
|     ISR|       Israel| 13.64%|
|     PAN|       Panama| 13.44%|
|     SVN|     Slovenia| 12.16%|
|     BEL|      Belgium| 11.78%|
|     ARM|      Armenia| 11.27%|
|     ESP|        Spain| 10.68%|
|     PRT|     Portugal| 10.46%|
+--------+-------------+-------+



#### Задание 2. 

Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [12]:
df_2 = df.filter(
    ((f.col('date') >= f.lit('2021-03-25')) & (f.col('date') <= f.lit('2021-03-31'))) & 
    ((~f.col('iso_code').contains('OWID')) | (f.col('iso_code').contains('KOS')))
).select([
    'location', 
    'new_cases', 
    'date']).withColumn('date', f.to_timestamp('date').cast('date'))

Даты отобраны

In [13]:
df_2.select(f.col('date').cast('date')).distinct().show(10)

+----------+
|      date|
+----------+
|2021-03-26|
|2021-03-25|
|2021-03-30|
|2021-03-27|
|2021-03-29|
|2021-03-28|
|2021-03-31|
+----------+



In [14]:
df_2.show()

+-----------+---------+----------+
|   location|new_cases|      date|
+-----------+---------+----------+
|Afghanistan|     34.0|2021-03-25|
|Afghanistan|     28.0|2021-03-26|
|Afghanistan|     36.0|2021-03-27|
|Afghanistan|      4.0|2021-03-28|
|Afghanistan|     28.0|2021-03-29|
|Afghanistan|     62.0|2021-03-30|
|Afghanistan|     70.0|2021-03-31|
|    Albania|    472.0|2021-03-25|
|    Albania|    449.0|2021-03-26|
|    Albania|    425.0|2021-03-27|
|    Albania|    493.0|2021-03-28|
|    Albania|    285.0|2021-03-29|
|    Albania|    304.0|2021-03-30|
|    Albania|    434.0|2021-03-31|
|    Algeria|    105.0|2021-03-25|
|    Algeria|    114.0|2021-03-26|
|    Algeria|     93.0|2021-03-27|
|    Algeria|     86.0|2021-03-28|
|    Algeria|    110.0|2021-03-29|
|    Algeria|    115.0|2021-03-30|
+-----------+---------+----------+
only showing top 20 rows



In [15]:
res_2 = df_2.withColumn('row_number',f.row_number().over(
    windowSpec.partitionBy('location').orderBy(f.col('new_cases').desc())
)).filter(
    f.col('row_number') == f.lit('1')).select([
    'location', 
    'new_cases', 
    'date']).orderBy(f.col(
    'new_cases').desc()).limit(10)

In [16]:
res_2.show()

+-------------+---------+----------+
|     location|new_cases|      date|
+-------------+---------+----------+
|       Brazil| 100158.0|2021-03-25|
|United States|  77321.0|2021-03-26|
|        India|  72330.0|2021-03-31|
|       France|  59054.0|2021-03-31|
|       Turkey|  39302.0|2021-03-31|
|       Poland|  35145.0|2021-03-26|
|      Germany|  25014.0|2021-03-31|
|        Italy|  24076.0|2021-03-26|
|         Peru|  19206.0|2021-03-25|
|      Ukraine|  18226.0|2021-03-26|
+-------------+---------+----------+



#### Задание 3  

Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [17]:
df_3 = df.filter(
    (f.col('date') >= f.lit('2021-03-25')) & 
    (f.col('date') <= f.lit('2021-03-31')) & 
    (f.col('iso_code') == f.lit('RUS'))
    ).select([
    'date', 
    'new_cases']).withColumn('date', f.to_timestamp('date').cast('date'))

In [18]:
df_3.show()

+----------+---------+
|      date|new_cases|
+----------+---------+
|2021-03-25|   9128.0|
|2021-03-26|   9073.0|
|2021-03-27|   8783.0|
|2021-03-28|   8979.0|
|2021-03-29|   8589.0|
|2021-03-30|   8162.0|
|2021-03-31|   8156.0|
+----------+---------+



In [19]:
df_3 = df_3.withColumn(
    'prev_day_cases', f.lag(f.col('new_cases'),1).over(windowSpec.partitionBy(f.lit(0)).orderBy('date')))

In [20]:
res_3 = df_3.withColumn('delta', f.col('new_cases') - f.col('prev_day_cases'))

In [21]:
res_3.show()

+----------+---------+--------------+------+
|      date|new_cases|prev_day_cases| delta|
+----------+---------+--------------+------+
|2021-03-25|   9128.0|          null|  null|
|2021-03-26|   9073.0|        9128.0| -55.0|
|2021-03-27|   8783.0|        9073.0|-290.0|
|2021-03-28|   8979.0|        8783.0| 196.0|
|2021-03-29|   8589.0|        8979.0|-390.0|
|2021-03-30|   8162.0|        8589.0|-427.0|
|2021-03-31|   8156.0|        8162.0|  -6.0|
+----------+---------+--------------+------+



In [22]:
!rm -rf res_1 res_2 res_3

In [23]:
res_1.write.option('header', 'true').csv('res_1')
res_2.write.option('header', 'true').csv('res_2')
res_3.write.option('header', 'true').csv('res_3')

# # JSON
# res_1.write.save('dataset_json', format='json')

# # Parquet
# res_1.write.save('dataset_parquet', format='parquet')